## Exemplos básicos de Engenharia de Atributos

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,  StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
dataset = pd.read_csv("C:/Users/Luiz Philipe Rangel/Dropbox/PC/Desktop/Curso Cientista de dados 2/CIENTISTADADOS2/Dados/2.Engenharia de Atributos/credito4.csv", sep=";")
#visulizar
dataset.shape

(1000, 8)

In [3]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [4]:
#separamos x de y
y = dataset['CLASSE']
X = dataset.iloc[:,0:7]

In [5]:
#Tratamento de valores faltantes
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [6]:
#preenche saldo atual pela mediana
mediana = X['SALDO_ATUAL'].median()
mediana

2323.0

In [7]:
X['SALDO_ATUAL'].fillna(mediana, inplace=True)
#checa resultado
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [8]:
#verifica qual a "moda" do estado civil
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [9]:
#substitui o estado civil pela moda
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
#checa resultado
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

In [10]:
#outliers, definimos como acima de 2 sd
desv = X['SALDO_ATUAL'].std()
desv

685936688.9820064

In [11]:
#checamos se algum SALDO_ATUAL atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ,'SALDO_ATUAL'] 

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [12]:
#vamos atualiar Saldo para mediana, calculamos
mediana =X['SALDO_ATUAL'].median()
mediana

2323.0

In [13]:
#atribumos
X.loc[X['SALDO_ATUAL'] >=  2 * desv, 'SALDO_ATUAL'] = mediana
#checamos se algum atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ] 

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


In [14]:
#data binding
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [16]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'obras', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'educação', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'qualificação', 'PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo             234
carro usado            103
mobilia/equipamento    181
negócios                97
outros                 105
radio/tv               280
dtype: int64

In [17]:
#extração de caracteristicas da data
#transforma coluna em data
X['DATA'] = pd.to_datetime(X['DATA'], format="%d/%m/%Y")

In [18]:
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [19]:
#criamos 3 novos atributos
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [20]:
X['DIASEMANA']

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIASEMANA, Length: 1000, dtype: object

In [21]:
#label encoder estado civil, proposito e dia semana
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [22]:
X['PROPOSITO'].unique()

array(['radio/tv', 'outros', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios'], dtype=object)

In [23]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [24]:
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])

In [25]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,5,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,5,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,2,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


In [26]:
#one hot encodigo
#OUTROSPLANOSPGTO
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [27]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')

In [28]:
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
995,0,1,0
996,0,1,0
997,0,1,0
998,0,1,0


In [29]:
#padronização Z-score
#gera objeto numpy
sc = StandardScaler()
m = sc.fit_transform( X.iloc[:,0:3])

In [30]:
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

In [31]:
#concatena os dados
X = pd.concat([X,z,pd.DataFrame(m, columns=['SALDO_ATUAL_N','RESIDENCIADESDE_N','IDADE_N'])], axis=1)

In [32]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,5,2019,1,5,0,1,0,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,5,2020,1,6,0,1,0,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,4,0,1,0,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,2,2019,1,6,0,1,0,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,0,1,0,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,2,2018,6,0,0,1,0,-0.543562,1.046987,-0.264338
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2,0,1,0,0.211898,1.046987,0.211566
997,804.0,4,38,nenhum,2018-07-03,3,5,2018,7,5,0,1,0,-0.875522,1.046987,0.105809
998,1845.0,4,23,nenhum,2019-07-04,3,5,2019,7,4,0,1,0,-0.504738,1.046987,-0.687363


In [33]:
#remover: saldo_atual e residenciadesde e idade foram normalizados
#outros planospagto foi feito onehot
#data foi extraida características
#OUTROS_banco por causa da dumy variable trap
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA', 'OUTROS_banco'], inplace=True)

In [34]:
X

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,3,5,2019,1,5,1,0,-0.745516,1.046987,1.639276
1,0,5,2020,1,6,1,0,0.957740,-0.765977,-0.740241
2,3,4,2020,1,4,1,0,-0.415337,0.140505,0.687469
3,3,2,2019,1,6,1,0,1.645526,1.046987,0.475956
4,3,0,2018,1,6,1,0,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...
995,0,2,2018,6,0,1,0,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,1,0,0.211898,1.046987,0.211566
997,3,5,2018,7,5,1,0,-0.875522,1.046987,0.105809
998,3,5,2019,7,4,1,0,-0.504738,1.046987,-0.687363


In [35]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,y,test_size = 0.3,random_state = 0)
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)

In [36]:
taxa_acerto

0.71